In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import argparse
import os

Using TensorFlow backend.


### **IMPORTANT NOTE:**
#### If you take a look at Figure you can start to see our training and validation loss start to rapidly divide. When you see training loss falling quickly while validation loss stagnates or even increases, you know you are overfitting.

![alt text](https://pyimagesearch.com/wp-content/uploads/2019/06/unfrozen.png)

## What Is **MobileNetV2**?
#### MobileNets are small, low-latency, low-power models parameterised to meet the resource constraints of a variety of use cases. According to the research paper, MobileNetV2 improves the state-of-the-art performance of mobile models on multiple tasks and benchmarks as well as across a spectrum of different model sizes.
#### MobileNetV2 is optimised for mobile devices. The architecture delivers high accuracy results while keeping the parameters and mathematical operations as low as possible to bring deep neural networks to mobile devices.
#### The weights of the pre-trained snippets were learned by the Google team using **ImageNet**.
#### NOTE: **Weights in MobileNetV2:** 

One of the following:
*   None (random initialization)
*   'imagenet' (pre-training on ImageNet)
*   or the path to the weights file to be loaded. Default to imagenet.

---
#### To accomplish **FACE MASK DETECTOR**, we’ll be fine-tuning the MobileNet V2 architecture, a highly efficient architecture that can be applied to embedded devices with limited computational capacity (ex., Raspberry Pi, Google Coral, NVIDIA Jetson Nano, etc.).




---

## Parsing a few command line arguments that are required to launch our script from a terminal

In [2]:
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
# 	help="path to input dataset")
# ap.add_argument("-p", "--plot", type=str, default="plot.png",
# 	help="path to output loss/accuracy plot")
# ap.add_argument("-m", "--model", type=str,
# 	default="mask_detector.model",
# 	help="path to output face mask detector model")
# args = vars(ap.parse_args())

## Defining Hyperparameters
#### We will be applying a learning rate decay schedule, which is why we’ve named the learning rate variable INIT_LR.

In [3]:
init_lr = 1e-4  # 0.0001
epochs = 20
bs = 32